# Install Packages

In [ ]:
!pip install -U pip
!pip install TTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 655.3/655.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 

In [ ]:
!pip install numba==0.50

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numba
  Running setup.py clean for numba
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llvmlite
  Running setup.py clean for llvmlite
Failed to build numba llvmlite
ER

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [ ]:
%env TRAINER_TELEMETRY=0

env: TRAINER_TELEMETRY=0


In [ ]:
output_path = "/content/gdrive/MyDrive/tts"


dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.txt", path=os.path.join(output_path, "NewTTSDataset/")
)

In [ ]:
audio_config = VitsAudioConfig(
    sample_rate=44100, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

In [ ]:
character_config = CharactersConfig(
    characters_class= "TTS.tts.models.vits.VitsCharacters",
    characters= "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890",
    punctuations=" !,.?-",
    pad= "<PAD>",
    eos= "<EOS>",
    bos= "<BOS>",
    blank= "<BLNK>",
)

In [ ]:
config = VitsConfig(
    audio=audio_config,
    characters=character_config,
    run_name="vits_vctk",
    batch_size=16,
    eval_batch_size=4,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=0,
    epochs=1000,
    text_cleaner="basic_cleaners",
    use_phonemes=False,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=False,
    save_best_after=1000,
    save_checkpoints=True,
    save_all_best=True,
    mixed_precision=True,
    max_text_len=250,  # change this if you have a larger VRAM than 16GB
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
    test_sentences=[
        ["Ini adalah tes suara"],
        ["Beberapa hal yang perlu diperhatikan ketika melatih sebuah model adalah parameter."],
        ["Selamat pagi"]
    ]
)

In [ ]:
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:44100
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [ ]:

def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "my_speaker"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = f"/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/{cols[0]}.wav"
            text = cols[1]
            # print(text)
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items

In [ ]:
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, formatter=formatter)

/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/2.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/3.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/4.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/5.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/6.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/7.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/8.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/9.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/10.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/11.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/12.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/13.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/14.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/15.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/16.wav exists
/content/gdrive/MyDrive/tts/NewTTSDataset/wavs/17.wav exists
/content/gdrive/MyDrive/tts/NewT

In [ ]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

In [ ]:
# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=/content/gdrive/MyDrive/tts/vits_vctk-May-06-2023_11+30AM-0000000

 > Model has 83047468 parameters


In [ ]:
trainer.fit()


 > EPOCH: 0/1000
 --> /content/gdrive/MyDrive/tts/vits_vctk-May-06-2023_11+30AM-0000000
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2023-05-06 11:30:24) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 349
 | > Preprocessing samples
 | > Max text length: 177
 | > Min text length: 58
 | > Avg text length: 128.31232091690543
 | 
 | > Max audio length: 441051.0
 | > Min audio length: 440597.0
 | > Avg audio length: 440924.99140401144
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


/usr/local/lib/python3.10/dist-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]

   --> STEP: 0/22 -- GLOBAL_STEP: 0
     | > loss_disc: 6.02148  (6.02148)
     | > loss_disc_real_0: 0.99595  (0.99595)
     | > loss_disc_real_1: 1.00790  (1.00790)
     | > loss_disc_real_2: 1.02794  (1.02794)
     | > loss_disc_real_3: 0.96881  (0.96881)
     | > loss_disc_real_4: 1.00997  (1.00997)
     | > loss_disc_real_5: 1.01039  (1.01039)
     | > loss_0: 6.02148  (6.02148)
     | > grad_norm_0: 0.00000  (0.00000)
     | > loss_gen: 6.01968  (6.01968)
     | > loss_kl: 15



> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 3
 | > Preprocessing samples
 | > Max text length: 173
 | > Min text length: 120
 | > Avg text length: 138.33333333333334
 | 
 | > Max audio length: 441051.0
 | > Min audio length: 440597.0
 | > Avg audio length: 440748.6666666667
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.


/usr/local/lib/python3.10/dist-packages/TTS/tts/models/vits.py:1454: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  test_figures["{}-alignment".format(idx)] = plot_alignment(alignment.T, output_fig=False)

  --> EVAL PERFORMANCE
     | > avg_loader_time: 2.04909 (+0.00000)
     | > avg_loss_disc: 2.55623 (+0.00000)
     | > avg_loss_disc_real_0: 0.26681 (+0.00000)
     | > avg_loss_disc_real_1: 0.17777 (+0.00000)
     | > avg_loss_disc_real_2: 0.21082 (+0.00000)
     | > avg_loss_disc_real_3: 0.22505 (+0.00000)
     | > avg_loss_disc_real_4: 0.10682 (+0.00000)
     | > avg_loss_disc_real_5: 0.11847 (+0.00000)
     | > avg_loss_0: 2.55623 (+0.00000)
     | > avg_loss_gen: 1.6

# Continue Training

After cloning the repo from coqui-tts, copy the customized `formatter` function to /TTS/TTS/bin/train_tts.py and set the `load_tts_sample` parameter to have `formatter=formatter` 

In [ ]:
!git clone https://github.com/coqui-ai/TTS.git

Cloning into 'TTS'...
remote: Enumerating objects: 29760, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 29760 (delta 98), reused 126 (delta 90), pack-reused 29607
Receiving objects: 100% (29760/29760), 159.17 MiB | 18.14 MiB/s, done.
Resolving deltas: 100% (21583/21583), done.


In [ ]:
!python /content/TTS/TTS/bin/train_tts.py --continue_path /content/gdrive/MyDrive/tts/vits_vctk-May-06-2023_11+41AM-0000000

Streaming output truncated to the last 5000 lines.
 | > Min text length: 58
 | > Avg text length: 128.31232091690543
 | 
 | > Max audio length: 441051.0
 | > Min audio length: 440597.0
 | > Avg audio length: 440924.99140401144
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.

 > TRAINING (2023-05-09 10:38:56) 

   --> STEP: 5/22 -- GLOBAL_STEP: 31450
     | > loss_disc: 2.30866  (2.41244)
     | > loss_disc_real_0: 0.13399  (0.12327)
     | > loss_disc_real_1: 0.25183  (0.21780)
     | > loss_disc_real_2: 0.24225  (0.22675)
     | > loss_disc_real_3: 0.24163  (0.21390)
     | > loss_disc_real_4: 0.21029  (0.16833)
     | > loss_disc_real_5: 0.21334  (0.20850)
     | > loss_0: 2.30866  (2.41244)
     | > grad_norm_0: 18.30391  (15.50426)
     | > loss_gen: 2.70555  (2.36927)
     | > loss_kl: 0.61585  (0.68737)
     | > loss_feat: 7.39550  (7.62103)
     | > loss_mel: 18.78860  (18.95851)
     | > loss_duration: 1.91141  (1.86297)
     | > amp_scaler: 64.00000  (64.000

# Inference

In [ ]:
!tts --text "soca adalah perusahaan rintisan di bidang teknologi informasi yang sedang berkembang, sekarang sedang fokus untuk melakukan pengembangan dibidang pengolahan suara dan pembuatan konten edukasi untuk seluruh rakyat indonesia" --model_path /content/gdrive/MyDrive/tts/vits_vctk-May-06-2023_11+41AM-0000000/best_model_32589.pth --config_path /content/gdrive/MyDrive/tts/vits_vctk-May-06-2023_11+41AM-0000000/config.json --out_path /content/tes_audio32589.mp3

 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:44100
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: soca adalah perusahaan rintisan di bidang teknologi informasi yang sedang berkembang, sekarang sedang fokus untuk melakukan pengembangan dibidang pengolahan suara dan pembuatan konten edukasi untuk seluruh rakyat indonesia
 >

In [ ]:
# function ConnectButton(){
#     console.log("Connect pushed"); 
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
# }
# setInterval(ConnectButton,60000);